### MINIMAL EXAMPLE OF JUPYTER NOTEBOOK WHICH CAN BE RUN WITH CROMWELL

### The problem is that it produces run2.html and trial_v1_movie_rec.gif in local folder

### IMPORT NECESSARY MODULES

In [6]:
#pip install matplotlib

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
import os
import logging
import sys
from datetime import datetime
import numpy as np
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1" #for debugging, it decrease performance dramatically

In [9]:
from utilities import *
from vae_model import * 

import torch
import pyro

# Set up pyro environment
pyro.clear_param_store()
pyro.set_rng_seed(0)

# Check versions
print("pyro.__version__  --> ",pyro.__version__)
print("torch.__version__ --> ",torch.__version__)
#assert(pyro.__version__.startswith('0.4'))
#assert(torch.__version__.startswith('1.2'))

pyro.__version__  -->  0.4.0
torch.__version__ -->  1.2.0


## read two json files and combine in single dictionary

In [10]:
params = load_json_as_dict("./input_for_wdl.json")  
params2 = load_json_as_dict("./other_inputs.json")
params.update(params2)
print(params)

{'run_ml_with_wdl.train_data': 'gs://ld-data-bucket/data/fashionmnist_train.pkl', 'run_ml_with_wdl.test_data': 'gs://ld-data-bucket/data/fashionmnist_test.pkl', 'run_ml_with_wdl.checkpoint': 'gs://ld-results-bucket/ckpt/dummy.pkl', 'run_ml_with_wdl.bucket_output': 'gs://ld-results-bucket/', 'run_ml_with_wdl.notebook_name': 'test.ipynb', 'run_ml_with_wdl.git_repo': 'https://github.com/dalessioluca/cromwell_for_ML.git', 'run_ml_with_wdl.commit_or_branch': 'master', 'simulation': {'__comment': 'there are 3 types of runs: scratch resume pre_trained', 'manual_or_wdl': 'wdl', 'output_dir': 'trial_v1', 'type': 'scratch'}, 'architecture': {'__comment': 'parameters specifying the architecture of the model', 'dim_zwhat': 25, 'width_input_image': 28, 'ch_input_image': 1}, 'loss': {'__comment': 'parameter of the observation model', 'mse_sigma': 0.1}, 'optimizer': {'__comment': 'which optimizer to use', 'type': 'adam', 'lr': 0.001, 'betas': [0.9, 0.999], 'eps': 1e-08}, 'training': {'__comment': 'pa

### prepare the file names

In [13]:
if params["simulation"]["manual_or_wdl"] == "manual":
    # the google bucket information is replaced by "./"
    # prepare input file and check if exist
    train_tmp = params["run_ml_with_wdl.train_data"].split("/")
    test_tmp = params["run_ml_with_wdl.test_data"].split("/")
    ckpt_tmp = params["run_ml_with_wdl.checkpoint"].split("/")
    train_file = os.path.join(train_tmp[-2],train_tmp[-1])
    test_file = os.path.join(test_tmp[-2],test_tmp[-1])
    ckpt_file = os.path.join(ckpt_tmp[-2],ckpt_tmp[-1])
    
    # prepare output files
    output_bucket = None
    output_dir = os.path.basename(params["simulation"]["output_dir"])
    try:
        os.mkdir(output_dir)
    except FileExistsError:
        pass

else:
    
    # At this point WDL has localized the files in the work directory
    # Therefore just grab basename
    train_file = os.path.basename(params["run_ml_with_wdl.train_data"])
    test_file = os.path.basename(params["run_ml_with_wdl.train_data"])
    ckpt_file = os.path.basename(params["run_ml_with_wdl.checkpoint"])
    
    # prepare output directory and check if exist
    output_bucket = params["run_ml_with_wdl.bucket_output"]
    output_dir = "./"

In [14]:
# checks
#assert os.path.isfile(train_file)
#assert os.path.isfile(test_file)
print(os.path.isfile(train_file))
print(os.path.isfile(test_file))
if params["simulation"]["type"] != "scratch":
    #assert os.path.isfile(ckpt_file)
    print(os.path.isfile(ckpt_file))
    
json_param_file = os.path.join(output_dir, "all_inputs.json")
log_file = os.path.join(output_dir, "jupyter.log")
    
print(output_bucket)
print(output_dir)
print(train_file)
print(test_file)
print(log_file)
print(json_param_file)

cur_dir = os.getcwd()
print(cur_dir)

None
trial_v1
data/fashionmnist_train.pkl
data/fashionmnist_test.pkl
trial_v1/jupyter.log
trial_v1/all_inputs.json
/Users/ldalessi/cromwell_for_ML


### start logging some data

In [8]:
logging.basicConfig(level=logging.INFO,
                    format="luca_logging: %(message)s",
                    filename=log_file,
                    filemode="w")
console = logging.StreamHandler()
formatter = logging.Formatter("luca_logging: %(message)s")
console.setFormatter(formatter)  # Use the same format for stdout.
logging.getLogger('').addHandler(console)  # Log to stdout and a file.

# Log the start time.
logging.info(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

luca_logging: 2020-02-28 17:27:33


In [40]:
logging.info("saving input json in output directory")
save_dict_as_json(params,json_param_file)

luca_logging: saving input json in output directory
